In [66]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

In [58]:

standings_awards = pd.read_csv('awards_playoffs.csv', index_col=[0])
thisyear_stats = pd.read_csv('thisyear_stats.csv', index_col = [0])

In [59]:
thisyear_stats.dropna(inplace=True)
thisyear_stats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 694 entries, 0 to 701
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Tm         694 non-null    object 
 1   Year       694 non-null    int64  
 2   QBOverall  694 non-null    float64
 3   WROverall  694 non-null    float64
 4   RBOverall  694 non-null    float64
 5   D_Overall  694 non-null    float64
dtypes: float64(4), int64(1), object(1)
memory usage: 38.0+ KB


In [60]:
standings_awards.info()

<class 'pandas.core.frame.DataFrame'>
Index: 702 entries, ARI to WAS
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   W-L%               702 non-null    float64
 1   Year               702 non-null    int64  
 2   R.O.D_record       702 non-null    float64
 3   MVPs               702 non-null    int64  
 4   OPOYs              702 non-null    int64  
 5   DPOYS              702 non-null    int64  
 6   ROY_OFFENSE        702 non-null    int64  
 7   ROY_DEFENSE        702 non-null    int64  
 8   QBProBowl          702 non-null    int64  
 9   ProBowl            702 non-null    int64  
 10  1stTeam            702 non-null    int64  
 11  2ndTeam            702 non-null    int64  
 12  Allpro             702 non-null    int64  
 13  Made Championship  702 non-null    int64  
 14  Made Superbowl     702 non-null    int64  
 15  Won SB             702 non-null    int64  
 16  Made Playoffs      702 non-nu

In [61]:
data = thisyear_stats.merge(standings_awards, how = 'outer', on = ['Tm', 'Year'])
data.dropna(inplace=True)
data.head()

,Tm,Year,QBOverall,WROverall,RBOverall,D_Overall,W-L%,R.O.D_record,MVPs,OPOYs,...,ROY_DEFENSE,QBProBowl,ProBowl,1stTeam,2ndTeam,Allpro,Made Championship,Made Superbowl,Won SB,Made Playoffs
0,ARI,2000,0.626506,0.757344,0.815789,0.080645,0.188,0.458333,0,0,...,0,0,1,0,0,0,0,0,0,0
1,ARI,2001,0.905405,0.701416,0.846154,0.161290,0.438,0.729333,0,0,...,0,0,1,1,0,1,0,0,0,0
2,ARI,2002,0.594595,0.550752,0.696429,0.062500,0.313,0.500333,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ARI,2003,0.601266,0.730216,0.702479,0.218750,0.250,0.604333,0,0,...,0,0,1,0,0,0,0,0,0,0
4,ARI,2004,0.666667,0.755319,0.825397,0.468750,0.375,0.396000,0,0,...,0,0,1,0,1,1,0,0,0,0


In [62]:
data.info()


<class 'pandas.core.frame.DataFrame'>
Index: 694 entries, 0 to 701
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Tm                 694 non-null    object 
 1   Year               694 non-null    int64  
 2   QBOverall          694 non-null    float64
 3   WROverall          694 non-null    float64
 4   RBOverall          694 non-null    float64
 5   D_Overall          694 non-null    float64
 6   W-L%               694 non-null    float64
 7   R.O.D_record       694 non-null    float64
 8   MVPs               694 non-null    int64  
 9   OPOYs              694 non-null    int64  
 10  DPOYS              694 non-null    int64  
 11  ROY_OFFENSE        694 non-null    int64  
 12  ROY_DEFENSE        694 non-null    int64  
 13  QBProBowl          694 non-null    int64  
 14  ProBowl            694 non-null    int64  
 15  1stTeam            694 non-null    int64  
 16  2ndTeam            694 non-null

In [63]:
data.describe()


,Year,QBOverall,WROverall,RBOverall,D_Overall,W-L%,R.O.D_record,MVPs,OPOYs,DPOYS,...,ROY_DEFENSE,QBProBowl,ProBowl,1stTeam,2ndTeam,Allpro,Made Championship,Made Superbowl,Won SB,Made Playoffs
count,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000,...,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000,694.000000
mean,2010.459654,0.765427,0.752398,0.849338,0.517651,0.501419,0.500282,0.033141,0.031700,0.031700,...,0.031700,0.276657,3.171470,0.755043,0.811239,1.566282,0.126801,0.063401,0.031700,0.383285
std,6.330755,0.167423,0.089764,0.115622,0.288389,0.192834,0.088716,0.179134,0.175327,0.175327,...,0.175327,0.447668,2.183037,1.073327,0.953669,1.571964,0.332990,0.243858,0.175327,0.486538
min,2000.000000,0.262500,0.451827,0.378882,0.031250,0.000000,0.250333,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2005.000000,0.653965,0.693169,0.785124,0.269531,0.375000,0.437667,0.000000,0.000000,0.000000,...,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2010.000000,0.794696,0.756502,0.871805,0.531250,0.500000,0.500333,0.000000,0.000000,0.000000,...,0.000000,0.000000,3.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
75%,2016.000000,0.901365,0.817683,0.938731,0.774194,0.625000,0.562667,0.000000,0.000000,0.000000,...,0.000000,1.000000,4.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,1.000000
max,2021.000000,1.000000,0.981221,1.000000,1.000000,1.000000,0.729333,1.000000,1.000000,1.000000,...,1.000000,1.000000,12.000000,6.000000,5.000000,9.000000,1.000000,1.000000,1.000000,1.000000


In [64]:
target = 'Made Playoffs'
features = [col for col in data.columns if col not in [target, 'Tm', 'Year', "Made Championship", "Made Superbowl","Won SB"]]

X = data[features]
y = data[target]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [67]:
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
}

gb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
}

xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
}

rf_model = RandomForestClassifier(random_state=42)
gb_model = GradientBoostingClassifier(random_state=42)
xgb_model = xgb.XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')

In [68]:
rf_grid_search = GridSearchCV(rf_model, rf_param_grid, cv=3, scoring='accuracy', n_jobs=-1)
rf_grid_search.fit(X_train_scaled, y_train)
rf_best_model = rf_grid_search.best_estimator_
rf_best_params = rf_grid_search.best_params_


In [69]:
rf_best_pred = rf_best_model.predict(X_test_scaled)
rf_best_accuracy = accuracy_score(y_test, rf_best_pred)
print("Random Forest Best Parameters:", rf_best_params)
print("Random Forest Accuracy:", rf_best_accuracy)

Random Forest Best Parameters: {'max_depth': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 200}
Random Forest Accuracy: 0.9280575539568345


In [70]:
print(classification_report(y_test, rf_best_pred))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94        86
           1       0.92      0.89      0.90        53

    accuracy                           0.93       139
   macro avg       0.93      0.92      0.92       139
weighted avg       0.93      0.93      0.93       139



In [71]:
gb_grid_search = GridSearchCV(gb_model, gb_param_grid, cv=3, scoring='accuracy', n_jobs=-1)
gb_grid_search.fit(X_train_scaled, y_train)
gb_best_model = gb_grid_search.best_estimator_
gb_best_params = gb_grid_search.best_params_

In [72]:
gb_best_pred = gb_best_model.predict(X_test_scaled)
gb_best_accuracy = accuracy_score(y_test, gb_best_pred)
print("Gradient Boosting Best Parameters:", gb_best_params)
print("Gradient Boosting Accuracy:", gb_best_accuracy)

Gradient Boosting Best Parameters: {'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
Gradient Boosting Accuracy: 0.9136690647482014


In [73]:
print(classification_report(y_test, gb_best_pred))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93        86
           1       0.89      0.89      0.89        53

    accuracy                           0.91       139
   macro avg       0.91      0.91      0.91       139
weighted avg       0.91      0.91      0.91       139



In [74]:
xgb_grid_search = GridSearchCV(xgb_model, xgb_param_grid, cv=3, scoring='accuracy', n_jobs=-1)
xgb_grid_search.fit(X_train_scaled, y_train)
xgb_best_model = xgb_grid_search.best_estimator_
xgb_best_params = xgb_grid_search.best_params_

/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [04:17:53] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [75]:
xgb_best_pred = xgb_best_model.predict(X_test_scaled)
xgb_best_accuracy = accuracy_score(y_test, xgb_best_pred)
print("XGBoost Best Parameters:", xgb_best_params)
print("XGBoost Accuracy:", xgb_best_accuracy)

XGBoost Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
XGBoost Accuracy: 0.9136690647482014


In [76]:
print(classification_report(y_test, xgb_best_pred))

              precision    recall  f1-score   support

           0       0.91      0.95      0.93        86
           1       0.92      0.85      0.88        53

    accuracy                           0.91       139
   macro avg       0.91      0.90      0.91       139
weighted avg       0.91      0.91      0.91       139

